In [2]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
pd.set_option('display.max_rows', 100)

import time
import os
import numpy as np
import pickle
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import plotly.express as px
from pmdarima.arima.utils import ndiffs, nsdiffs
from pmdarima.arima import auto_arima, ARIMA
from pmdarima import model_selection
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [7]:
api_key = "6e68642c-8403-4caa-af31-bda40b8c67f6" # web token for RESTful API
country_code = "10Y1001A1001A83F" # Germany
time_zone = "Europe/Berlin" # time zone for Germany
start_date = "20180101"
end_date = "20180601"
start_date_2 = "20180601"
end_date_2 = "20180901"
downsample = False

In [8]:
class SARIMADataset:
    """
    Fetch and preprocess ENTSO-E load and generation data to fit SARIMA model.
    """
    def __init__(
        self,
        start_date, 
        end_date,                          # overall end date
        api_key,                           # web token for RESTful API
        country_code = "10Y1001A1001A83F", # country code (default: Germany)
        time_zone = "Europe/Berlin",       # time zone for Germany
        downsample = False                 # downsample 1h resolution
                ):
        """
        Initialize SARIMA dataset.
        Params
        ------
        start_date : str
                     overall start date as "YYYYMMDD"
        end_date : str
                   overall end date as "YYYYMMDD"
        api_key : str
                  web token for RESTfulAPI access to ENTSO-E transparency platform
        country_code : str
                       country code
        time zone : str
                    time zone
        downsample : bool
                     Downsample to 1h resolution or not.
        """
        self.start_date = start_date
        self.end_date = end_date
        self.country = country_code
        self.time_zone = time_zone
        self.api_key = api_key
        self.df = None
        self.original_headers = None
        self.errors = None
        self.models = None
        self.downsample = downsample
        self.fetch_data()
        self.calculate_pslps()
        #self.calculate_errors()
    
    def _get_load_intervals(self):
        """
        Get time points for sequential data loading from ENTSO-E transparency platform.
        
        For one request, the time delta for loading data from the platform is limited to one year.
        
        
        Returns
        -------
        pd.Series
        pandas series with timestamps of time points to consider between start and end date
        """
        # Convert start and end dates to timestamps.
        start = pd.Timestamp(self.start_date, tz=self.time_zone)
        end = pd.Timestamp(self.end_date, tz=self.time_zone)
    
        # Create series from start and end timestamps.
        start_series = pd.Series(pd.Timestamp(self.start_date))
        end_series = pd.Series(pd.Timestamp(self.end_date))
        
        # Create date range from start and end dates and determine year starts within range.
        # Convert data range to series.
        dates = pd.date_range(start=self.start_date, end=self.end_date, freq="YS", inclusive="both").to_series()
    
        # Check whether start date itself is year start.
        # If not, prepend to dates to consider for data loading.
        if not start.is_year_start:
            dates = pd.concat([start_series, dates], ignore_index=True)
    
        # Check whether end date itself is year start.
        # If not, append to dates to consider for data loading.
        if not end.is_year_start:
            dates = pd.concat([dates, end_series], ignore_index=True)
            
        return dates
        
        
    def _load_data(self, start_date, end_date):
        """
        Load actual load and actual aggregated generation per production type for requested time interval.
        Params
        ------
        start_date : str
                     start date as "yyyymmdd"
        end_date : str
                   end date as "yyyymmdd"
                    
        Returns
        -------
        pd.DataFrame with time points as indices and load + generation per type as columns.
        """
        from entsoe import EntsoePandasClient
        # Initialize client and settings.
        client = EntsoePandasClient(api_key=self.api_key)
        start = pd.Timestamp(start_date, tz=self.time_zone)
        end = pd.Timestamp(end_date, tz=self.time_zone)
        # Query data and save to dataframe.
        df_load = client.query_load(self.country, start=start, end=end)
        print(f"Actual load has shape {df_load.shape}.")
        df_gen = client.query_generation(self.country, start=start, end=end, psr_type=None)
        df_gen.columns = [" ".join(a) for a in df_gen.columns.to_flat_index()]
        print(f"Actual generation per production type has shape {df_gen.shape}.")
        df_final = pd.concat([df_load, df_gen], axis=1) # Concatenate dataframes in columns dimension.
        print(f"Concatenated data frame has shape {df_final.shape}.")
        
        return df_final

    
    def fetch_data(self, drop_consumption=True):
        """
        Fetch actual load and generation per type from ENTSO-E transparency platform 
        for requested time interval. Set resulting dataframe as attribute.
        
        Parameters
        ----------
        drop_consumption : Bool
                           Drop columns containing actual consumption.
        """
        # Determine sequence of dates to consider when loading data.
        dates = self._get_load_intervals()
        print(f"Consider the following dates:\n{dates}")
        df_list = []
        
        for i, _ in enumerate(dates):
    
            if i == dates.shape[0] - 1:
                df_final = pd.concat(df_list, axis=0) # Concatenate dataframes along time axis (index).
                df_final.index = pd.to_datetime(df_final.index, utc=True).tz_convert(tz="UTC+01:00")
    
                # Drop columns containing actual consumption?
                if drop_consumption:
                    print("Dropping columns containing actual consumption...")
                    df_final.drop(list(df_final.filter(regex='Consumption')), axis=1, inplace=True)
                original_headers = df_final.columns

                print("Creating columns for PSLP calculation...")
                for header in original_headers:
                    df_final[str(header) + " PSLP"] = pd.Series(dtype='float')
                if self.downsample:
                    print("Downsample to 1h resolution...")
                    df_final = df_final.resample('1H', axis='index').mean()
                print("Returning final data frame...")
                self.df = df_final
                self.original_headers = original_headers
                return
                
            try:
                print(f"Trying to load data chunk for time interval [{dates[i]}, {dates[i+1]}]...")
                df_temp = self._load_data(start_date=dates[i], end_date=dates[i+1])
                print(df_temp.shape)
                df_list.append(df_temp)
                print("Loading successful!")
                
            except Exception as e:
                print(f"Loading failed!", e)
                continue

    @staticmethod            
    def get_pslp_category(date, weekday=None, holiday=None, country_code='DE'):
        """
        Get PSLP category from date, weekday information, and holiday information.
        0 : weekday
        1 : Saturday
        2 : Sunday and holiday
        
        Params
        ------
        date : str
               date in 'YYYYMMDD' format
        weekday : int
                  corresponding weekday
                  0 - Mon, 1 - Tue, 2 - Wed, 3 - Thu, 4 - Fri, 5 - Sat, 6 - Sun
        holiday : Bool
                  True if public holiday, False if not.
        
        Returns
        -------
        int : PSLP category
        """
        # Convert string-type date to datetime object.
        if type(date) is str:
            date = pd.to_datetime(date)
        
        # Assign weekday if not given.
        if weekday is None:
            weekday = date.weekday()
        
        # Assign holiday category if not given.
        if holiday is None:
            import holidays
            holiday = date in holidays.country_holidays(country_code)
        
        # Special treatment for Christmas eve and New year's eve as Saturdays.
        if ( date.day == 24 or date.day == 31 ) and date.month == 12 and weekday != 6:
            pslp_category = 1
        # weekdays
        elif weekday < 5 and holiday is False:
            pslp_category = 0
        # Saturdays
        elif weekday == 5 and holiday is False:
            pslp_category = 1
        # Sundays and holidays
        elif weekday == 6 or holiday is True:
            pslp_category = 2
        return pslp_category
    
    
    def _assign_pslp_categories(self, country_code="DE"):
        """
        Assign PSLP categories to dates in dataframe's datetime index.
        Amemd dataframe by weekday information, holiday information, and PSLP category

        0 is weekday, 1 is Saturday, 2 is Sunday or holiday.
        Special treatment for Christmas eve and New Year's eve (as Saturdays).
    
        Params
        ------
        df : pandas.Dataframe
        country_code : str
                       country to determine holidays for
        """
    
        import holidays
        
        # Get holidays in specified country.
        country_holidays = holidays.country_holidays(country_code) # Passing a state is also possible!
    
        s = self.df.index.to_series()                           # Convert datetime index to series.
        dates = s.dt.date                                       # Get plain dates from datetime objects.
        weekdays = s.dt.weekday                                 # Get weekdays from datetime objects.
        holidays = [date in country_holidays for date in dates] # Determine holidays.
        pslp_category = []
        
        for d, wd, hd in zip(dates, weekdays, holidays):
            pslp_category.append(self.get_pslp_category(d, wd, hd))
            
        self.df["PSLP Category"] = pslp_category
        self.df["Holiday"] = holidays
        self.df["Weekday"] = weekdays
        return

    
    def _calculate_pslp(self, date_str, lookback=3, country_code='DE', DEBUG=False):
        """
        Calculate PSLP for given date from given data.
        
        The data is categorized into weekdays, Saturdays, and Sundays/holidays.
        The `lookback` most recent days from the specified date's category are used to
        calculate the corresponding PSLP as the average.
        
        Params
        ------
        date_str : str
                   date 'YYYYMMDD' to calculate PSLP for; if None, calculate PSLP for all dates
        lookback : int
                   number of days to consider in each category for calculating PSLP
        country_code : str
                       considered country (for holidays)
        
        """
        unique_dates = self.df.index.to_series().dt.date.drop_duplicates().tolist() # Get unique dates in data index.
        self._assign_pslp_categories(country_code)
    
        if DEBUG:
            print(f"Calculating PSLPs for date {date_str}...")
        date = pd.to_datetime(date_str)
        
        pslp_category = self.get_pslp_category(date_str)
        if DEBUG:
            print(f"PSLP category of {date.date()} is {pslp_category}.")
    
        # Check whether date is in range of given dataframe.
        if date.date() < unique_dates[0]:
            raise IndexError(f"PSLP cannot be calculated. Date {date_str} is in the past.")
        if date.date()  > unique_dates[-1] + pd.Timedelta(days = 1) and date.date() != _get_nearest_future_pslp_date(date_str, pslp_category):
            raise IndexError(f"PSLP cannot be calculated. Date {date_str} is too far in the future.")
        assert date.date() in unique_dates
        
        unique_dates_pslp = self.df[self.df['PSLP Category']==pslp_category].index.to_series().dt.date.drop_duplicates().tolist()
        idx_pslp = unique_dates_pslp.index(date.date())
        if DEBUG:
            print(f"Index in unique days of PSLP category is {idx_pslp}.")
        if idx_pslp - lookback < 0:
            raise IndexError(f"PSLP cannot be calculated. Less than {lookback} samples in PSLP category for date {date_str}.")
        lookback_dates = [pd.to_datetime(d).strftime('%Y-%m-%d') for d in unique_dates_pslp[idx_pslp-lookback:idx_pslp]]
        if DEBUG:
            print(f"Dates to consider for calculating PSLP: {lookback_dates}")
        for header in self.original_headers:
            if DEBUG:
                print(f"{header}...")
            pslp_temp = pd.concat([self.df[header].at[d].reset_index(drop=True) for d in lookback_dates], axis=1).mean(axis=1)
            #print(self.df[header+" PSLP"].at[date_str].shape, pslp_temp.shape)
            num_points = self.df[header].at[date_str].shape[0]
            self.df[header+" PSLP"].at[date_str] = pslp_temp.head(num_points)
            

    def calculate_pslps(self, date_str=None, lookback=3, country_code='DE', DEBUG=False):
        """
        Calculate PSLPs for all dates in dataframe or for given date from given data.
        
        The data is categorized into weekdays, Saturdays, and Sundays/holidays.
        The `lookback` most recent days from the specified date's category are used to
        calculate the corresponding PSLP as the average.
        
        Params
        ------
        df : pandas.Dataframe
             data to calculate PSLP for, must have datetime index
        original_headers : list of str
                           categories to calculate PSLP for
        date_str : str
                   date 'YYYYMMDD' to calculate PSLP for; if None, calculate PSLP for all dates
        lookback : int
                   number of days to consider in each category for calculating PSLP
        country_code : str
                       considered country (for holidays)
        
        """
        if date_str is not None:
            print(f"Calculating PSLP for date {date_str} only...")
            self._calculate_pslp(date_str, 
                                 lookback=lookback, 
                                 country_code=country_code, 
                                 DEBUG=DEBUG)
        
        else:
            print("Calculating PSLPs for all dates in dataframe...")
            unique_dates = self.df.index.to_series().dt.date.drop_duplicates().tolist() # Get unique dates in data index.
            unique_dates = [pd.to_datetime(d).strftime('%Y-%m-%d') for d in unique_dates]
            
            for date in tqdm(unique_dates):
                try:
                    self._calculate_pslp(date_str=date, 
                                         lookback=lookback, 
                                         country_code=country_code, 
                                         DEBUG=DEBUG)
                except IndexError as e:
                    print(e)
                    

    def calculate_residuals(self):
        """
        Calculate residuals of actual data w.r.t PSLPs.
        """
        for header in self.original_headers:
            self.df[header+" Residuals"] = self.df[header] - self.df[header+" SARIMA"]
    
    
    def plot_data(self):
        """
        Plot preprocessed load and generation data.
        """
        from plotly.subplots import make_subplots
        import plotly.graph_objects as go
    
        num_rows = len(self.original_headers)
        
        fig = make_subplots(rows=num_rows, cols=1, subplot_titles=(self.original_headers))
    
        for i, header in enumerate(self.original_headers):
            fig.add_trace(go.Scatter(x = self.df.index, y = self.df[header], name=header), row=i+1, col=1)
            fig.add_trace(go.Scatter(x = self.df.index, y = self.df[header+" PSLP"], name=header+" PSLP"), row=i+1, col=1)
            fig.add_trace(go.Scatter(x = self.df.index, y = self.df[header+" Residuals"], name=header+" Residuals"), row=i+1, col=1)
    
        fig.update_layout(height=10000, width=1200)
        fig.show()
        
       
    def calculate_errors(self):
        """
        Calculate forecasting errors for preprocessed ENTSO-E load and generation data.  
        """
        from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
        errors = {}
        for header in self.original_headers:
            temp = pd.concat([self.df[header], self.df[header+" PSLP"]], axis=1).dropna()
            mae = mean_absolute_error(temp[header], temp[header+" PSLP"])
            mape = mean_absolute_percentage_error(temp[header], temp[header+" PSLP"])
            mse = mean_squared_error(temp[header], temp[header+" PSLP"])
            errors[header] = {"MAE": mae, "MAPE": mape, "MSE": mse}
        self.errors = errors
        return
    
    
    def fit_sarima(self, 
                   save_dir,
                   update_level,
                   load_dir=None,
                   min_p = 1,
                   min_q = 1,
                   max_p = 3,
                   max_q = 3,
                   min_P = 1,
                   min_Q= 1,
                   max_P = 2,
                   max_Q = 2):
        """
        Fit SARIMA model to data.
        
        Params
        ------
        save_dir : str
                   path to root dir where to save models to
        update_level : int
                       0 - update current model with new observations
                       1 - refit model of known (seasonal) order
                       2 - find completely new model with `auto_arima`
        load_dir : str
                   path to root dir where to load models from
        min_p : int
        min_q : int
        max_p : int
        max_q : int
        min_P : int
        min_Q : int
        max_P : int
        max_Q : int
        """
        # Determine seasonal cycle for SARIMA model.
        # Each day is a season, i.e., 24 * 1h or 96 * 15min.
        seasonal_cycle = 24 if self.downsample else 96
        
        models = []
        
        # Check if specified load dir exists.
        if load_dir is not None:
            if os.path.isdir(load_dir) is False:
                print("WARNING: Specified loading directory does not exist. Fitting models from scratch.")
                update_level = 2
        
        os.makedirs(save_dir, exist_ok=True)

        if update_level == 0:
            print("Update existing models with new observations...")
            # Loop over categories in data, i.e., load and generation types.
            for header in self.original_headers:
                print(f"Update model for {header}.")
                start_time = time.perf_counter()
                with open(load_dir+f"arima_{header}.pkl".replace(" ", "_").replace("/", "_"), "rb") as pkl:
                    model = pickle.load(pkl)
                model.update(self.df[header])
                duration = time.perf_counter() - start_time
                print(f"DONE: Updating model for {header} took {duration} s.")
                fname = save_dir+f"arima_{header}.pkl".replace(" ", "_").replace("/", "_")
                with open(fname, 'wb') as pkl:
                    pickle.dump(model, pkl)
                models.append(model)
                print(model.summary())
                          
        elif update_level == 1:
            print("Refit models of known (seasonal) order on given data...")
            # Loop over categories in data, i.e., load and generation types.
            for header in self.original_headers:
                print(f"Refit model for {header}.")
                start_time = time.perf_counter()
                with open(load_dir+f"arima_{header}.pkl".replace(" ", "_").replace("/", "_"), "rb") as pkl:
                    old_model = pickle.load(pkl)
                model = ARIMA(order=old_model.order, seasonal_order=old_model.seasonal_order)
                model.fit(self.df[header])
                duration = time.perf_counter() - start_time
                print(f"DONE: Refitting model for {header} took {duration} s.")
                fname = save_dir+f"arima_{header}.pkl".replace(" ", "_").replace("/", "_")
                with open(fname, 'wb') as pkl:
                    pickle.dump(model, pkl)
                models.append(model)
                print(model.summary())
                
        elif update_level == 2:
            print("Fit new models from scratch with `auto_arima`...")
            # Loop over categories in data, i.e., load and generation types.
            for header in self.original_headers:
                print(f"Consider {header}.")
                print("Pre-compute (seasonal) differencing order to accelerate auto-ARIMA...")
                # Estimate order of differencing d by performing a stationarity test for different d's. 
                # Selects max. value d for which time series is judged stationary by statistical test.
                # Default unit root test of stationarity: Kwiatkowski–Phillips–Schmidt–Shin (KPSS)
                start_time = time.perf_counter()
                d = ndiffs(self.df[header], test='kpss')
                # Estimate order of seasonal differencing D by performing stationarity test of seasonality for different D's. 
                # Selects max. value D for which time series is judged seasonally stationary by statistical test.
                # Default unit root test of stationarity: Osborn-Chui-Smith-Birchenhall (OCSB)
                D = nsdiffs(self.df[header], m=seasonal_cycle, test='ocsb')
                print(f"Differencing order is {d}. Seasonal differencing order is {D}.")
                print(f"Automatically discover optimal order for SARIMAX model for {header}...")
                model = auto_arima(self.df[header], 
                                   start_p=min_p, 
                                   d=d, 
                                   start_q=min_q,
                                   max_p=max_p,
                                   max_q=max_q,
                                   start_P=min_P,
                                   start_Q=min_Q,
                                   max_P=max_P,
                                   max_Q=max_Q,
                                   D=D, 
                                   m=seasonal_cycle,
                                   trace=True
                                  )
                duration = time.perf_counter() - start_time
                print(f"DONE: Fitting model for {header} from scratch took {duration} s.")

                models.append(model)
                fname = save_dir+f"arima_{header}.pkl".replace(" ", "_").replace("/", "_")
                with open(fname, 'wb') as pkl:
                    pickle.dump(model, pkl)
                print(model.summary())
                
        self.models = models
        print("DONE.")
        return
    
    def predict(self):
        pass
    
    def forecast(self, date):
        #if date=None:
        #    date = date.today()
        date = pd.Timestamp(date, tz="UTC+01:00") #- timedelta(days=days_to_subtract)
        day = pd.Timedelta(days=1)
        halfyear = pd.Timedelta(weeks=21)

In [9]:
SarimaData = SARIMADataset(start_date, end_date, api_key, country_code, time_zone, downsample=downsample)

Consider the following dates:
0   2018-01-01
1   2018-06-01
dtype: datetime64[ns]
Trying to load data chunk for time interval [2018-01-01 00:00:00, 2018-06-01 00:00:00]...
Actual load has shape (14492, 1).
Actual generation per production type has shape (14492, 24).
Concatenated data frame has shape (14492, 25).
(14492, 25)
Loading successful!
Dropping columns containing actual consumption...
Creating columns for PSLP calculation...
Returning final data frame...
Calculating PSLPs for all dates in dataframe...


  0%|          | 0/151 [00:00<?, ?it/s]

PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-01.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-02.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-03.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-04.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-06.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-07.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-13.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-14.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2018-01-20.


ValueError: cannot set using a slice indexer with a different length than the value

In [5]:
print(len(SarimaData.original_headers), SarimaData.original_headers)

18 Index(['Actual Load', 'Biomass Actual Aggregated',
       'Fossil Brown coal/Lignite Actual Aggregated',
       'Fossil Coal-derived gas Actual Aggregated',
       'Fossil Gas Actual Aggregated', 'Fossil Hard coal Actual Aggregated',
       'Fossil Oil Actual Aggregated', 'Geothermal Actual Aggregated',
       'Hydro Pumped Storage Actual Aggregated',
       'Hydro Run-of-river and poundage Actual Aggregated',
       'Hydro Water Reservoir Actual Aggregated', 'Nuclear Actual Aggregated',
       'Other Actual Aggregated', 'Other renewable Actual Aggregated',
       'Solar Actual Aggregated', 'Waste Actual Aggregated',
       'Wind Offshore Actual Aggregated', 'Wind Onshore Actual Aggregated'],
      dtype='object')


In [ ]:
SarimaData.fit_sarima(save_dir="save_2/", update_level=2)

Fit new models from scratch with `auto_arima`...
Consider Actual Load.
Pre-compute (seasonal) differencing order to accelerate auto-ARIMA...
Differencing order is 1. Seasonal differencing order is 0.
Automatically discover optimal order for SARIMAX model for Actual Load...
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(1,0,1)[24] intercept   : AIC=59807.039, Time=8.69 sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=67043.531, Time=0.04 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=60580.907, Time=3.12 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=62248.965, Time=3.22 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=67041.532, Time=0.03 sec
 ARIMA(1,1,1)(0,0,1)[24] intercept   : AIC=61453.525, Time=3.34 sec
 ARIMA(1,1,1)(1,0,0)[24] intercept   : AIC=60280.596, Time=4.96 sec
 ARIMA(1,1,1)(2,0,1)[24] intercept   : AIC=inf, Time=29.63 sec
 ARIMA(1,1,1)(1,0,2)[24] intercept   : AIC=59628.285, Time=31.11 sec
 ARIMA(1,1,1)(0,0,2)[24] intercept   : AIC=60928.053, Time=17.76 sec
 ARIMA

/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(3,1,2)(2,0,0)[24] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/

 ARIMA(3,1,2)(2,0,0)[24] intercept   : AIC=inf, Time=nan sec
 ARIMA(2,1,1)(2,0,0)[24]             : AIC=51889.925, Time=11.14 sec
 ARIMA(2,1,1)(1,0,0)[24]             : AIC=52415.310, Time=2.68 sec
 ARIMA(2,1,1)(2,0,1)[24]             : AIC=inf, Time=36.71 sec


/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,1)(1,0,1)[24]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/

 ARIMA(2,1,1)(1,0,1)[24]             : AIC=inf, Time=nan sec
 ARIMA(1,1,1)(2,0,0)[24]             : AIC=52230.867, Time=9.43 sec
 ARIMA(2,1,0)(2,0,0)[24]             : AIC=inf, Time=13.62 sec
 ARIMA(3,1,1)(2,0,0)[24]             : AIC=inf, Time=31.34 sec
 ARIMA(2,1,2)(2,0,0)[24]             : AIC=inf, Time=26.20 sec
 ARIMA(1,1,0)(2,0,0)[24]             : AIC=inf, Time=10.16 sec
 ARIMA(1,1,2)(2,0,0)[24]             : AIC=52005.838, Time=13.03 sec
 ARIMA(3,1,0)(2,0,0)[24]             : AIC=inf, Time=15.37 sec
 ARIMA(3,1,2)(2,0,0)[24]             : AIC=inf, Time=43.69 sec

Best model:  ARIMA(2,1,1)(2,0,0)[24]          
Total fit time: 683.549 seconds
                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                 3623
Model:             SARIMAX(2, 1, 1)x(2, 0, [], 24)   Log Likelihood              -25938.962
Date:                             Wed, 15 Mar 2023   AIC            

In [7]:
def plot_autocorrelations(data):
    """Plot ACF and PACF for given data."""
    plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
    # Original series
    fig, axes = plt.subplots(3, 3)
    axes[0, 0].plot(data.to_numpy()); axes[0, 0].set_title(f'Original Series')
    plot_acf(data.to_numpy(), ax=axes[0, 1])
    plot_pacf(data.to_numpy(), ax=axes[0, 2], method='ywm')

    # 1st-order differencing
    axes[1, 0].plot(data.diff().to_numpy()); axes[1, 0].set_title(f'1st-Order Differencing')
    plot_acf(data.diff().dropna().to_numpy(), ax=axes[1, 1])
    plot_pacf(data.to_numpy(), ax=axes[1, 2], method='ywm')

    # 2nd-order differencing
    axes[2, 0].plot(data.diff().diff().to_numpy()); axes[2, 0].set_title(f'2nd-Order Differencing')
    plot_acf(data.diff().diff().dropna().to_numpy(), ax=axes[2, 1])
    plot_pacf(data.to_numpy(), ax=axes[2, 2], method='ywm')
    plt.tight_layout()
    plt.show()

In [21]:
#data_test = SarimaData.df["Actual Load"].loc["2018-01-01"]
#print(data_test)
#plot_autocorrelations(data_test)

In [9]:
def split_train_test(df, train_fraction=0.75, seasonal_cycle=96):
    """
    Perform season-based train-test split of input data.
    Params
    ------
    df : pandas.DataFrame
         overall dataset
    train_fraction : float
                     fraction of data to use for training
    seasonal_cycle : int
                     number of data points in each season
                     Default 96 corresponds to 15min time resolution.
    
    Returns
    -------
    df_train : pandas.DataFrame
               train dataset
    df_test : pandas.DataFrame
              test dataset
    """
    overall_days = int(SarimaData.df.shape[0] / seasonal_cycle)
    train_days = int(train_fraction * overall_days)
    test_days = overall_days - train_days
    return model_selection.train_test_split(df, train_size=train_days, test_size=test_days)

In [22]:
#train_fraction = 0.75
#data = SarimaData.df["Actual Load"]
#data_train, data_test = split_train_test(data, train_fraction=train_fraction, seasonal_cycle=seasonal_cycle)

In [18]:
print(delta)
type(model)
print(model.__dir__())
print(model.summary())
# Serialize model.
with open('sarima.pkl', 'wb') as pkl:
    pickle.dump(model, pkl)

# You can still make predictions from the model at this point
print(model.predict(n_periods=5))

-713.5144805535674
['order', 'seasonal_order', 'start_params', 'method', 'maxiter', 'suppress_warnings', 'out_of_sample_size', 'scoring', 'scoring_args', 'trend', 'with_intercept', 'sarimax_kwargs', 'endog_index_', 'arima_res_', 'fit_with_exog_', 'nobs_', 'pkg_version_', 'oob_', 'oob_preds_', '__module__', '__doc__', '__init__', '_fit', 'fit', '_check_exog', 'predict_in_sample', 'predict', '__getstate__', '__setstate__', '_warn_for_older_version', '__str__', 'update', 'aic', 'aicc', 'arparams', 'arroots', 'bic', 'bse', 'conf_int', 'df_model', 'df_resid', 'fittedvalues', 'hqic', 'maparams', 'maroots', 'oob', 'params', 'pvalues', 'resid', 'summary', 'to_dict', 'plot_diagnostics', '__abstractmethods__', '_abc_impl', 'fit_predict', '_get_param_names', 'get_params', 'set_params', '__repr__', '_more_tags', '_get_tags', '_check_n_features', '_check_feature_names', '_validate_data', '_validate_params', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '__dict__', '__weakref__', '__hash__

In [11]:
with open('arima.pkl', 'rb') as pkl:
    pickle_preds = pickle.load(pkl).predict(n_periods=5)
print(pickle_preds)

2018-01-02 00:00:00+01:00    43907.815550
2018-01-02 01:00:00+01:00    42584.228530
2018-01-02 02:00:00+01:00    42405.341563
2018-01-02 03:00:00+01:00    43692.797580
2018-01-02 04:00:00+01:00    44813.506766
Freq: H, dtype: float64


In [12]:
print(SarimaData.original_headers)

Index(['Actual Load', 'Biomass Actual Aggregated',
       'Fossil Brown coal/Lignite Actual Aggregated',
       'Fossil Coal-derived gas Actual Aggregated',
       'Fossil Gas Actual Aggregated', 'Fossil Hard coal Actual Aggregated',
       'Fossil Oil Actual Aggregated', 'Geothermal Actual Aggregated',
       'Hydro Pumped Storage Actual Aggregated',
       'Hydro Run-of-river and poundage Actual Aggregated',
       'Hydro Water Reservoir Actual Aggregated', 'Nuclear Actual Aggregated',
       'Other Actual Aggregated', 'Other renewable Actual Aggregated',
       'Solar Actual Aggregated', 'Waste Actual Aggregated',
       'Wind Offshore Actual Aggregated', 'Wind Onshore Actual Aggregated'],
      dtype='object')


In [6]:
SarimaData2 = SARIMADataset(start_date_2, end_date_2, api_key, country_code, time_zone, downsample=downsample)

Consider the following dates:
0   2018-06-01
1   2018-09-01
dtype: datetime64[ns]
Trying to load data chunk for time interval [2018-06-01 00:00:00, 2018-09-01 00:00:00]...
Actual load has shape (8832, 1).
Actual generation per production type has shape (8832, 24).
Concatenated data frame has shape (8832, 25).
(8832, 25)
Loading successful!
Dropping columns containing actual consumption...
Creating columns for PSLP calculation...
Downsample to 1h resolution...
Returning final data frame...


In [13]:
SarimaData2.fit_sarima(save_dir="save_1/", update_level=1, load_dir="save_2/")

Refit models of known (seasonal) order on given data...
Refit model for Actual Load.


/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                        SARIMAX Results                                        
Dep. Variable:                                       y   No. Observations:                 5137
Model:             SARIMAX(0, 1, 3)x(1, 0, [1, 2], 24)   Log Likelihood              -41518.685
Date:                                 Wed, 15 Mar 2023   AIC                          83053.370
Time:                                         19:01:48   BIC                          83105.723
Sample:                                     05-31-2018   HQIC                         83071.694
                                          - 12-31-2018                                         
Covariance Type:                                   opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -6.4959      1.560     -4.165      0.000      -9.553      -3

/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                 5137
Model:             SARIMAX(2, 1, 1)x(2, 0, [], 24)   Log Likelihood              -19736.644
Date:                             Wed, 15 Mar 2023   AIC                          39487.288
Time:                                     19:05:26   BIC                          39533.096
Sample:                                 05-31-2018   HQIC                         39503.321
                                      - 12-31-2018                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.1229      0.294     -0.419      0.676      -0.699       0.453
ar.L1         -0.2978      

...

                                        SARIMAX Results                                        
Dep. Variable:                                       y   No. Observations:                 5137
Model:             SARIMAX(2, 1, 3)x(0, 0, [1, 2], 24)   Log Likelihood                -291.142
Date:                                 Wed, 15 Mar 2023   AIC                            600.285
Time:                                         19:07:04   BIC                            659.181
Sample:                                     05-31-2018   HQIC                           620.899
                                          - 12-31-2018                                         
Covariance Type:                                   opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0060      5.409     -0.001      0.999     -10.607      10

/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                        SARIMAX Results                                        
Dep. Variable:                                       y   No. Observations:                 5137
Model:             SARIMAX(1, 1, 1)x(1, 0, [1, 2], 24)   Log Likelihood              -24182.750
Date:                                 Wed, 15 Mar 2023   AIC                          48379.501
Time:                                         19:08:40   BIC                          48425.309
Sample:                                     05-31-2018   HQIC                         48395.534
                                          - 12-31-2018                                         
Covariance Type:                                   opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0054      0.029      0.184      0.854      -0.052       0

/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                      SARIMAX Results                                       
Dep. Variable:                                    y   No. Observations:                 5137
Model:             SARIMAX(3, 1, 2)x(1, 0, [1], 24)   Log Likelihood              -39025.784
Date:                              Wed, 15 Mar 2023   AIC                          78069.568
Time:                                      19:08:56   BIC                          78128.464
Sample:                                  05-31-2018   HQIC                         78090.182
                                       - 12-31-2018                                         
Covariance Type:                                opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -9.4907      2.879     -3.296      0.001     -15.134      -3.847
ar.L1         -0.09

In [7]:
SarimaData3 = SARIMADataset(start_date_2, end_date_2, api_key, country_code, time_zone, downsample=downsample)
SarimaData3.fit_sarima(save_dir="save_3/", update_level=0, load_dir="save_2/")

Consider the following dates:
0   2018-06-01
1   2018-09-01
dtype: datetime64[ns]
Trying to load data chunk for time interval [2018-06-01 00:00:00, 2018-09-01 00:00:00]...
Actual load has shape (8832, 1).
Actual generation per production type has shape (8832, 24).
Concatenated data frame has shape (8832, 25).
(8832, 25)
Loading successful!
Dropping columns containing actual consumption...
Creating columns for PSLP calculation...
Downsample to 1h resolution...
Returning final data frame...
Update existing models with new observations...
Update model for Actual Load.
                                        SARIMAX Results                                        
Dep. Variable:                                       y   No. Observations:                 5831
Model:             SARIMAX(0, 1, 3)x(1, 0, [1, 2], 24)   Log Likelihood              -47043.227
Date:                                 Thu, 16 Mar 2023   AIC                          94102.455
Time:                                       

 **Regarding the warnings:**   
The first one is actually more like a "note" than a "warning". It's just letting you know how the covariance matrix was computed.  
The second one is letting you know that parameter estimates may be unstable. Sometimes this is an indication of overfitting, but it can also arise from other things. This may indicate that you should try a simpler model (which then might forecast better), but it does not mean that you have to do that.


In [10]:
print(SarimaData3.models)

[ARIMA(order=(0, 1, 3), scoring_args={}, seasonal_order=(1, 0, 2, 24),
      suppress_warnings=True), ARIMA(order=(0, 1, 1), scoring_args={}, seasonal_order=(2, 0, 0, 24),
      suppress_warnings=True, with_intercept=False), ARIMA(order=(2, 1, 3), scoring_args={}, seasonal_order=(2, 0, 0, 24),
      suppress_warnings=True, with_intercept=False), ARIMA(order=(2, 1, 3), scoring_args={}, seasonal_order=(0, 0, 0, 24),
      suppress_warnings=True, with_intercept=False), ARIMA(order=(2, 1, 0), scoring_args={}, seasonal_order=(2, 0, 0, 24),
      suppress_warnings=True, with_intercept=False), ARIMA(order=(2, 1, 0), scoring_args={}, seasonal_order=(2, 0, 0, 24),
      suppress_warnings=True, with_intercept=False), ARIMA(order=(3, 1, 1), scoring_args={}, seasonal_order=(2, 0, 0, 24),
      suppress_warnings=True, with_intercept=False), ARIMA(order=(1, 1, 1), scoring_args={}, seasonal_order=(2, 0, 0, 24),
      suppress_warnings=True, with_intercept=False), ARIMA(order=(2, 0, 0), scoring_args={

In [15]:
date = "20220101"
def predict(date_str, root=None, downsample=True):
    if root is None or os.exists(root) is False:
        root = "./"
    forecast_horizon = 24 if downsample else 96
    end = pd.Timestamp(date_str, tz="UTC+01:00")
    middle = end - pd.Timedelta(days=1)
    start = end - pd.Timedelta(weeks=21)
    print(type(start))
    print(start, middle, end)
    base_set = SARIMADataset(start.strftime('%Y%m%d'), middle.strftime('%Y%m%d'), api_key, country_code, time_zone, downsample=downsample)
    base_set.fit_sarima(save_dir=root+f"base_{date}/", update_level=2)
    update_set = SARIMADataset(middle.strftime('%Y%m%d'), end.strftime('%Y%m%d'), api_key, country_code, time_zone, downsample=downsample)
    update_set.fit_sarima(load_dir=root+f"base_{date}/", save_dir=root+f"update_{date}/", update_level=0)
    forecasts = []
    for header, model in zip(update_set.original_headers, update_set.models):
        forecast = model.predict(n_periods=forecast_horizon)
        forecasts.append(forecast)
    return forecasts

In [20]:
forecasts = predict(date)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2021-08-07 00:00:00+01:00 2021-12-31 00:00:00+01:00 2022-01-01 00:00:00+01:00
Consider the following dates:
0   2021-08-07
1   2021-12-31
dtype: datetime64[ns]
Trying to load data chunk for time interval [2021-08-07 00:00:00, 2021-12-31 00:00:00]...
Actual load has shape (14020, 1).
Actual generation per production type has shape (14020, 20).
Concatenated data frame has shape (14020, 21).
(14020, 21)
Loading successful!
Dropping columns containing actual consumption...
Creating columns for PSLP calculation...
Downsample to 1h resolution...
Returning final data frame...
Calculating PSLPs for all dates in dataframe...


  0%|          | 0/147 [00:00<?, ?it/s]

PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-06.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-07.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-08.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-09.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-10.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-14.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-15.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-21.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-08-22.
Fit new models from scratch with `auto_arima`...
Consider Actual Load.
Pre-compute (seasonal) differencing order to accelerate auto-ARIMA...
Differencing order is 1. Seasonal differencing order is 0.
Automatically discover optimal orde

/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(1,1,1)(1,0,1)[24] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/

 ARIMA(1,1,1)(1,0,1)[24] intercept   : AIC=inf, Time=nan sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=38942.396, Time=0.05 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=inf, Time=2.59 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=35313.221, Time=2.64 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=38940.403, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[24] intercept   : AIC=36812.145, Time=0.26 sec
 ARIMA(0,1,1)(1,0,1)[24] intercept   : AIC=inf, Time=5.13 sec
 ARIMA(0,1,1)(0,0,2)[24] intercept   : AIC=34561.503, Time=11.77 sec
 ARIMA(0,1,1)(1,0,2)[24] intercept   : AIC=inf, Time=25.71 sec
 ARIMA(0,1,0)(0,0,2)[24] intercept   : AIC=35505.679, Time=12.82 sec
 ARIMA(1,1,1)(0,0,2)[24] intercept   : AIC=34360.256, Time=14.98 sec
 ARIMA(1,1,1)(0,0,1)[24] intercept   : AIC=34957.131, Time=3.79 sec
 ARIMA(1,1,1)(1,0,2)[24] intercept   : AIC=inf, Time=60.62 sec
 ARIMA(1,1,0)(0,0,2)[24] intercept   : AIC=34423.898, Time=11.27 sec
 ARIMA(2,1,1)(0,0,2)[24] intercept   : AIC=33580.801, Time=37.55 sec
 ARI

...

  0%|          | 0/1 [00:00<?, ?it/s]

PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2021-12-31.
Update existing models with new observations...
Update model for Actual Load.
DONE: Updating model for Actual Load took 8.100497052073479 s.
                                        SARIMAX Results                                        
Dep. Variable:                                       y   No. Observations:                 3529
Model:             SARIMAX(3, 1, 0)x(1, 0, [1, 2], 24)   Log Likelihood              -28608.088
Date:                                 Thu, 16 Mar 2023   AIC                          57232.176
Time:                                         15:25:44   BIC                          57281.524
Sample:                                              0   HQIC                         57249.780
                                                - 3529                                         
Covariance Type:                                   opg                                         
      

/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/tsa/statespace/mlemodel.py:2979: RuntimeWarning: divide by zero encountered in divide
  return self.params / self.bse
/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/tsa/stattools.py:1346: RuntimeWarning: invalid value encountered in divide
  test_statistic = numer_squared_sum / denom_squared_sum
/hkfs/home/project/hk-project-test-mss/ku4408/.virtualenvs/entsoe/lib64/python3.9/site-packages/statsmodels/tsa/stattools.py:681: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]


                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                 3529
Model:             SARIMAX(1, 1, 1)x(1, 0, [], 24)   Log Likelihood                   0.000
Date:                             Thu, 16 Mar 2023   AIC                              8.000
Time:                                     15:26:50   BIC                             32.674
Sample:                                          0   HQIC                            16.802
                                            - 3529                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.9990         -0        inf      0.000      -0.999      -0.999
ma.L1          0.9691      

ValueError: Input contains NaN.